<a href="https://colab.research.google.com/github/lamcnguyen89/CAP_6411_Assignments/blob/main/Assignment_04/Assignment_04_Mistral_and_Mixtral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 4: Mistral vs Mixtral Pipeline**



*   Got to https://people.eecs.berkeley.edu/~hendrycks/data.tar
*   Colab should have both mixtral and mistral 7b pipelines
*   Compare both in terms of speed and accuracy
*   Bonus: 100 points for applying SMoE to other models
*   Due 03Sep2024





**Mistral-7b** \
Mistral-7B is one of the biggest and most advanced Large Language Models out there, trained on a massive dataset of text and code.

Link: https://huggingface.co/mistralai/Mistral-7B-v0.1/discussions/13

**Mixtral 8x7b** \
Mixtral 8x7b is a high-quality sparse mixture of experts (SMoE) model with open weights created by Mistral AI. It outperforms Llama 2 70B on most benchmarks and batches or beats GPT3.5 on most standard benchmarks.

Link: https://github.com/dvmazur/mixtral-offloading/blob/master/notebooks/demo.ipynb


# **Load Mistral-7B Model and Test**

In [1]:
# Download Mistral-7B Model
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install bitsandbytes accelerate xformers einops
!pip -q install langchain
!pip install --upgrade huggingface_hub

# You need to go onto Google Colab and create a variable in Secrets. This variable will contain a token for the huggingface account that we will get our pretrained models from
from google.colab import userdata
secret = userdata.get('HF_TOKEN')
!huggingface-cli login --token {secret}





  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!nvidia-smi

Mon Sep  2 23:53:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              50W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.set_default_device('cuda')

In [4]:
# Import Pretrained model and load into memory
# Note I required a GPU with 40gb of memory for this project.
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             torch_dtype="auto")

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                          torch_dtype="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
# How to Prompt Mistral AI Models: https://community.aws/content/2dFNOnLVQRhyrOrMsloofnW0ckZ/how-to-prompt-mistral-ai-models-and-why


text = "<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST]"

encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

device = 'cuda'
model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<s>[INST] What is your favourite condiment? [/INST]Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Certainly! Here's a simple recipe for homemade mayonnaise that you can try out:

Ingredients:

* 1 cup mayonnaise
* 1 egg yolk
* 1 tablespoon Dijon mustard
* 1 garlic clove, minced
* 1/4 cup champagne vinegar or white wine vinegar
* 1/2 teaspoon salt
* 1/4 teaspoon black pepper
* 3 tablespoons vegetable oil

Instructions:

1. In a medium-sized mixing bowl, whisk together the egg yolk, dijon mustard, minced garlic, and vinegar until well combined.
2. Slowly whisk in the vegetable oil, continue whisking until the mixture thickens and is a pale yellow in color.
3. Season with salt and pepper to taste.
4. Cover the bowl with plastic wrap and refrigerate for at least an hour before using.

Note: It's best to use fresh ingredients for the b

# **Load and Process Dataset**


In [ ]:
# Source: https://github.com/mddunlap924/PyTorch-LLM/blob/main/notebooks/training.ipynb

